In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf

In [8]:
model = tf.keras.models.load_model('best_model.keras')
print("Model loaded successfully.")

Model loaded successfully.


In [3]:
train_path = 'Brain_Tumor/Training'
categories = os.listdir(train_path)

In [4]:
test_path = 'Brain_Tumor/Testing'
X_test = []
y_test = []

In [5]:
for category in categories:
    sub_dir_path = os.path.join(test_path, category)
    images = os.listdir(sub_dir_path)
    for img in images:
        img_path = os.path.join(sub_dir_path, img)
        img_array = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img_array is None:
            print(f"Failed to load image: {img_path}")
            continue
        img_resize = cv2.resize(img_array, (150, 150))
        img_normalized = img_resize.astype('float32') / 255.0
        img_processed = np.expand_dims(img_normalized, axis=-1)  # Shape: (150, 150, 1)
        X_test.append(img_processed)
        y_test.append(categories.index(category))
    print('Loaded', category)

Loaded glioma
Loaded meningioma
Loaded notumor
Loaded pituitary


In [6]:
X_test = np.array(X_test)
y_test = np.array(y_test)

In [9]:
loss,accuracy = model.evaluate(X_test,y_test)
print("LOSS :",loss)
print("Accuracy : ",accuracy)

41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - accuracy: 0.7404 - loss: 0.6581
LOSS : 0.48895496129989624
Accuracy :  0.8176963925361633


In [ ]:
# Define the image preprocessing function
def preprocess_image(image_path, img_size=(150, 150)):
    """
    Reads an image from the given path, converts it to grayscale,
    resizes it to the specified dimensions, normalizes the pixel values,
    and adds the required channel dimension.
    """
    # Read the image in grayscale
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"Could not read the image from {image_path}")
    
    # Resize the image to match the model's expected input size
    img_resized = cv2.resize(img, img_size)
    
    # Normalize the image to the [0, 1] range
    img_normalized = img_resized.astype('float32') / 255.0
    
    # Add a channel dimension (since the model expects shape: (height, width, channels))
    img_processed = np.expand_dims(img_normalized, axis=-1)
    
    # Add a batch dimension: (1, height, width, channels)
    img_processed = np.expand_dims(img_processed, axis=0)
    
    return img_processed

In [11]:
# Provide the path to the image you want to test
test_image_path = r'Brain_Tumor\Testing\notumor\Te-no_0169.jpg'  

# Preprocess the image
try:
    processed_image = preprocess_image(test_image_path)
except ValueError as e:
    print(e)
    exit()
# Make a prediction using the model
predictions = model.predict(processed_image)
predicted_class = np.argmax(predictions, axis=1)[0]
confidence = np.max(predictions)

print(f"Predicted Class: {predicted_class}")
print(f"Confidence: {confidence:.2f}")


class_names = categories  
if predicted_class < len(class_names):
    print(f"Predicted Label: {class_names[predicted_class]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Predicted Class: 2
Confidence: 1.00
Predicted Label: notumor


In [12]:
test_list = ['Brain_Tumor/Testing/pituitary/Te-pi_0012.jpg','Brain_Tumor/Testing/glioma/Te-gl_0016.jpg','Brain_Tumor/Testing/meningioma/Te-me_0021.jpg','Brain_Tumor/Testing/notumor/Te-no_0199.jpg']

In [13]:
for path in test_list:
    test_image_path = path  # Replace with your actual image path
    actual_class = path.split('/')[2]
    # Preprocess the image
    try:
        processed_image = preprocess_image(test_image_path)
    except ValueError as e:
        print(e)
        exit()
    # Make a prediction using the model
    predictions = model.predict(processed_image)
    predicted_class = np.argmax(predictions, axis=1)[0]
    confidence = np.max(predictions)

    print(f"Predicted Class: {predicted_class}")
    print(f"Confidence: {confidence:.2f}")

    # Optionally, if you have a mapping of class indices to names, you can display that as well.
    # For example:
    class_names = ['glioma', 'meningioma', 'notumor', 'pituitary']  # Replace with your actual class names
    if predicted_class < len(class_names):
        print(f"Predicted Label: {class_names[predicted_class]}")
        print(f"actual class : {actual_class}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Predicted Class: 3
Confidence: 0.91
Predicted Label: pituitary
actual class : pituitary
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Predicted Class: 0
Confidence: 0.97
Predicted Label: glioma
actual class : glioma
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Class: 2
Confidence: 0.51
Predicted Label: notumor
actual class : meningioma
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Predicted Class: 2
Confidence: 1.00
Predicted Label: notumor
actual class : notumor
